# Wordle Solver

In [1]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

In [2]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = "as"
green_positions = [3, 5]
yellow_letters = ""
yellow_positions = []
bad_letters = "crnehilt"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters, green_positions, yellow_letters, yellow_positions, bad_letters
)
print(words)

['amass']


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      A      2
      S      2
      M      1


#### Choosing next word

In [4]:
# Main function that returns a list of possible words
green_letters = ""
green_positions = []
yellow_letters = "slthi"
yellow_positions = []
bad_letters = ""

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    answer_word_list=False,
)
print(words)

['hilts', 'liths']


In [5]:
df = wordle.load_data()
df

,Names,Games_Won
0,Murilo,257
1,Barbara,108
2,Draw,258


In [6]:
# Update score
score_df = wordle.score()
score_df

,Names,Games_Won
0,Murilo,258
1,Barbara,108
2,Draw,258


In [7]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        258
Barbara        108
   Draw        258


In [8]:
# help(wordle.reset_score)

#### Set custom score

In [9]:
# help(wordle.set_score)

In [10]:
# # Set custom score
# df = wordle.set_score(m_score=253, b_score=105, draw_score=252)
# df

#### Reset score

In [11]:
## Reset score
# wordle.reset_score()

In [12]:
def can_construct(word, s):
    """Check if the word can be constructed from the string while keeping the order."""
    it = iter(s)
    return all(char in it for char in word)


def filter_constructible_words(word_list, s):
    """Return the list of words that can be constructed from the string s."""
    return [word for word in word_list if can_construct(word, s)]


# Example usage
word_list = ["apple", "app", "peach", "plea", "leap", "banana"]
s = "applppleaechbn"

# This will return ['apple', 'app', 'plea', 'leap']
result = filter_constructible_words(word_list, s)
result

['apple', 'app', 'peach', 'plea']

In [13]:
def can_construct(word, s):
    for i in s:
        if i in word:
            return True


def filter(word_list, s):
    words = []
    for word in word_list:
        if can_construct(word, s):
            words.append(word)
    return word


words = filter(word_list, s)
words

'banana'

In [14]:
def word_break(s, lexicon):
    n = len(s)
    dp = [0] * (n + 1)  # Probability of best segmentation
    prev = [-1] * (n + 1)  # Backtracking for reconstruction
    dp[0] = 1  # Base case: empty string has a segmentation with probability 1

    def is_valid_word(word):
        return word in lexicon

    def get_probability(words):
        # Placeholder: Should return the language model's probability of the sequence of words
        return 1.0

    # Fill dp table
    for i in range(1, n + 1):
        for j in range(i):
            word = s[j:i]
            if is_valid_word(word) and dp[j] > 0:
                prob = dp[j] * get_probability(word)
                if prob > dp[i]:
                    dp[i] = prob
                    prev[i] = j

    # Reconstruct the best segmentation
    if dp[n] == 0:
        return None  # No valid segmentation
    words = []
    i = n
    while i > 0:
        words.append(s[prev[i] : i])
        i = prev[i]
    words.reverse()

    return words


# Example usage
lexicon = {"a", "is", "test", "thesis", "this"}
sequence = "thisisatest"
result = word_break(sequence, lexicon)
print(result)

['this', 'is', 'a', 'test']


In [15]:
def threeSum(nums: list[int]) -> list[list[int]]:
    output = []
    distinct = []
    nums.sort()
    i, j, k = 0, 1, 2
    for i in range(0, len(nums) - 2):
        for j in range(1, len(nums) - 1):
            for k in range(2, len(nums)):
                list_nums = [nums[i], nums[j], nums[k]]
                sum_nums = sum(list_nums)
                if i != j and i != k and j != k and sum_nums == 0:
                    if set(list_nums) not in distinct:
                        distinct.append(set(list_nums))
                        output.append(list_nums)
    return output

In [16]:
nums = [-1, 0, 1, 2, -1, -4]
threeSum(nums=nums)

[[-1, -1, 2], [-1, 0, 1]]

In [17]:
def isPalindrome(s: str) -> bool:
    pal = ""
    for char in s.lower():
        if char.isalnum():
            pal += char
    for i in range(len(pal)):
        if pal[i] == pal[-i - 1]:
            continue
        else:
            return False
    return True

In [18]:
s = "A man, a plan, a canal: Panama"
ans = isPalindrome(s)
print(ans)

True


In [19]:
def binary_search(arr, target):
    assert target <= len(arr) - 1
    low, high = 0, len(arr) - 1
    mid = 0

    while low <= high:
        mid = (low + high) // 2
        if arr[mid] < target:
            low = mid + 1
        elif arr[mid] > target:
            high = mid - 1
        else:
            return mid
    return -1

In [20]:
arr = [0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 15, 17, 20]
ans = binary_search(arr, target=3)
print(arr)
print(ans)

[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 15, 17, 20]
-1


In [21]:
def calPoints(operations: list[str]) -> int:
    output = []
    for op in range(len(operations)):
        try:
            if type(int(operations[op])):
                new_score = int(operations[op])
        except:
            pass
        if operations[op] == "D":
            new_score = output[-1] * 2
        if operations[op] == "C":
            output.pop()
            continue
        if operations[op] == "+":
            new_score = output[-1] + output[-2]
        output.append(new_score)
    return sum(output)

In [22]:
ops = ["5", "2", "C", "D", "+"]
calPoints(ops)

30

In [23]:
def isValid(s: str) -> bool:
    stack = []
    mapping = {"(": ")", "[": "]", "{": "}"}
    for i in range(len(s)):
        if s[i] in mapping:
            stack.append(s[i])
        else:
            if len(stack) and mapping[stack[-1]] == s[i]:
                stack.pop()
            else:
                return False
    return stack == []

In [24]:
s = "]"
isValid(s)

False

In [25]:
class MinStack:
    def __init__(self):
        self.arr = []
        self.min_stack = []

    def push(self, val: int) -> None:
        self.arr.append(val)
        if not self.min_stack or val <= self.min_stack[-1]:
            self.min_stack.append(val)

    def pop(self) -> None:
        if self.arr:
            if self.arr[-1] == self.min_stack[-1]:
                self.min_stack.pop()
            self.arr.pop()

    def top(self) -> int:
        if self.arr:
            return self.arr[-1]
        return None

    def getMin(self) -> int:
        if self.min_stack:
            return self.min_stack[-1]
        return None

In [26]:
minstack = MinStack()
minstack.push(0)
minstack.arr

[0]

In [35]:
def math_challenge(num1, num2):
    while num2 != 0:
        num1, num2 = num2, num1 % num2
    return num1


# Example usage
num1 = 12
num2 = 16
print(math_challenge(num1, num2))

4


In [32]:
def ArrayChallenge(arr):
    sort_arr = sorted(arr)
    max1, max2 = sort_arr[-1], sort_arr[-2]
    prod = max1 * max2
    sum_arr = sum(arr) * 2
    if prod > sum_arr:
        return "true"
    return "false"

In [5]:
def ArrayChallenge(arr):
    # Initialize the two largest numbers
    if len(arr) < 2:
        return "false"  # Not enough elements to form a pair
    max1 = float("-inf")
    max2 = float("-inf")
    # Initialize the sum
    total_sum = 0
    for num in arr:
        # Update the total sum
        total_sum += num
        # Update the two largest numbers
        if num > max1:
            max2 = max1
            max1 = num
        elif num > max2:
            max2 = num
        print(max1, max2)
    # Calculate the product of the two largest numbers
    product = max1 * max2
    # Calculate double the total sum
    double_sum = 2 * total_sum
    # Compare the product with the doubled sum
    if product > double_sum:
        return "true"
    return "false"

In [6]:
# Example usage
arr = [2, 5, 6, -6, 26, 2, 3, 6, 5, 3]
print(ArrayChallenge(arr))

2 -inf
5 2
6 5
6 5
26 6
26 6
26 6
26 6
26 6
26 6
true
